In [16]:
import pandas as pd
import googlemaps

In [17]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [18]:
# Read in data
df = pd.read_csv('MuskMap - Sheet1.csv')

In [19]:
df['full_address'] = df['Street'] + ' ' + df['City'] + ' ' + df['State']

In [20]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [21]:
# Extract latitude and longitude using tuple indexing
df['latitude'] = df['geocoded'].apply(lambda loc: loc[0] if loc else None)
df['longitude'] = df['geocoded'].apply(lambda loc: loc[1] if loc else None)

# Check for any failed geocoding
if df['latitude'].isnull().any():
    print("Some addresses could not be geocoded. Please check the addresses.")

print(df[['full_address', 'latitude', 'longitude']])

                                full_address   latitude  longitude
0         2200 Caldwell Lane Del Valle Texas  30.217206 -97.544015
1                  1 Tesla Road Austin Texas  30.228074 -97.616477
2                   858 FM1209 Bastrop Texas  30.154372 -97.406265
3         130 Walker Watson Rd Bastrop Texas  30.153044 -97.404438
4                   865 FM1209 Bastrop Texas  30.155268 -97.403317
5         126 Walker-Watson Rd Bastrop Texas  30.152409 -97.404225
6           1 Rocket Road Brownsville  Texas  25.987613 -97.186418
7  50520 Rio Grande Drive Brownsville  Texas  25.964820 -97.208075
8  40998 Quick Silver Ave Brownsville  Texas  25.959815 -97.227827


In [22]:
# Drop the 'geocoded' column as it's no longer needed
df.drop(columns=['geocoded'], inplace=True)

In [24]:
import geopandas as gpd
from shapely.geometry import Point
import folium
# from folium.plugins import MarkerCluster

# Drop rows where geocoding failed (optional, based on your needs)
df_geo = df.dropna(subset=['latitude', 'longitude']).copy()

# Create geometry column
df_geo['geometry'] = df_geo.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df_geo, geometry='geometry')

# Set Coordinate Reference System (CRS) to WGS84 (latitude and longitude)
gdf.set_crs(epsg=4326, inplace=True)

# Optionally, export to GeoJSON
gdf.to_file("data.geojson", driver='GeoJSON')

# Initialize the map centered around the mean coordinates
map_center = [df_geo['latitude'].mean(), df_geo['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Initialize MarkerCluster
# marker_cluster = MarkerCluster().add_to(m)

# Function to create HTML for popups
def create_popup(row):
    if row['NewName'] != row['OldName']:
        name_html = f"<span style='text-decoration: line-through; color: red;'>Old Name: {row['OldName']}</span><br><span style='color: green;'>Musk Name: {row['NewName']}</span>"
    else:
        name_html = row['OldName']
    
    # Combine all information with styling
    popup_html = f"""
    <div style="font-family: Arial, sans-serif; font-size: 14px;">
        <b>{name_html}</b><br>
        <b>Address:</b> {row['full_address']}<br>
        <b>Description:</b> {row['Name']}
    </div>
    """
    return folium.Popup(popup_html, max_width=300)

# Add markers to the map
for idx, row in df_geo.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=create_popup(row),
#         tooltip=row['OldName'] if row['NewName'] == row['OldName'] else row['OldName']
    ).add_to(m)

# Save the map to an HTML file
m.save("index.html")
print("Interactive map has been saved to 'index.html'.")
m

Interactive map has been saved to 'index.html'.


In [9]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MuskMap
